In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import IFrame
InteractiveShell.ast_node_interactivity = "all"

# Companies House x HMRC exporters

I want to build this in a way that one can improve a link pair in a notebook, then deploy those changes to the link easily. I'm going to play with this idea here.

In [2]:
from src.data import utils as du
from src.models import utils as mu
from src.config import tables, stopwords
from src.features.clean_complex import clean_comp_names
from src.link.make_link import LinkDatasets

from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl

# import os
import logging
import mlflow
from functools import partial
from dotenv import load_dotenv, find_dotenv

In [3]:
settings = {
    "link_type": "link_only",
    "unique_id_column_name": "dim_uuid",
    "retain_matching_columns": False,
    "retain_intermediate_calculation_columns": False,
    "blocking_rules_to_generate_predictions": [
        """
            (l.name_unusual_tokens = r.name_unusual_tokens)
            and (
                l.name_unusual_tokens <> ''
                and r.name_unusual_tokens <> ''
            )
        """,
        """
            (l.postcode = r.postcode)
            and (
                l.postcode <> ''
                and r.postcode <> ''
            )
        """,
        """
            (l.name_unusual_tokens_first5 = r.name_unusual_tokens_first5)
            and (
                length(l.name_unusual_tokens_first5) = 5
                and length(r.name_unusual_tokens_first5) = 5
            )
        """,
        """
            (l.name_unusual_tokens_last5 = r.name_unusual_tokens_last5)
            and (
                length(l.name_unusual_tokens_last5) = 5
                and length(r.name_unusual_tokens_last5) = 5
            )
        """
    ],
    "comparisons": [
        cl.jaro_winkler_at_thresholds(
            "name_unusual_tokens", [0.9, 0.6], term_frequency_adjustments=True
        ),
        ctl.postcode_comparison("postcode")
    ],
}

In [4]:
pipeline = {
    "estimate_probability_two_random_records_match": {
        "deterministic_matching_rules": """
            l.name_unusual_tokens = r.name_unusual_tokens
        """,
        "recall": 0.7
    },
    "estimate_u_using_random_sampling": {
        "max_pairs": 1e6
    },
    "estimate_parameters_using_expectation_maximisation": {
        "blocking_rule": """
            l.name_unusual_tokens = r.name_unusual_tokens
        """
    },
    "estimate_parameters_using_expectation_maximisation": {
        "blocking_rule": """
            l.postcode = r.postcode
        """
    }
}

In [5]:
ch_settings = {
    "name": '"companieshouse"."companies"',
    "select": [
        "id::text as dim_uuid",
        "company_name",
        "postcode"
    ],
    "preproc": {
        clean_comp_names: {
            "primary_col": "company_name",
            "secondary_col": None,
            "stopwords": stopwords
        }
    }
}

In [6]:
exp_settings = {
    "name": '"hmrc"."trade__exporters"',
    "select": [
        "dim_uuid::text",
        "company_name",
        "postcode"
    ],
    "preproc": {
        clean_comp_names: {
            "primary_col": "company_name",
            "secondary_col": None,
            "stopwords": stopwords
        }
    }
}

In [7]:
ch_x_exp = LinkDatasets(
    table_l = ch_settings,
    table_r = exp_settings,
    settings = settings,
    pipeline = pipeline
)

In [8]:
ch_x_exp.get_data()

/opt/conda/envs/company_matching/lib/python3.9/site-packages/pandas/io/sql.py:1410: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  meta = MetaData(self.connectable, schema=schema)


In [9]:
ch_x_exp.preprocess_data()

In [10]:
ch_x_exp.pipeline = pipeline

In [1]:
ch_x_exp.create_linker()
ch_x_exp.train_linker()

NameError: name 'ch_x_exp' is not defined

In [ ]:
ch_x_exp.linker.estimate_parameters_using_expectation_maximisation(
    blocking_rule = """
        l.postcode = r.postcode
    """
)

In [ ]:
predictions = ch_x_exp.linker.predict(threshold_match_probability=0.7) 

In [ ]:
predictions.as_pandas_dataframe.head(10)

In [ ]:
existing = du.dataset(ch_x_exp.pair['eval'])